In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as cts
import pylcp
import time
from bayes_opt import BayesianOptimization
from scipy.optimize import root_scalar

In [2]:
Gamma = 22 # Hz to kHz, Decay rate
wavelength = 359.3e-9 # m to mm
k = 1/wavelength*2*np.pi #x_0
x0 = 1/k
t0 = 1/Gamma*1/(2*np.pi*1e6)
v0 = x0/t0
m0 = cts.hbar*t0/x0**2
a0 = x0/t0**2
F0 = cts.hbar/(x0*t0)
mass = 43*cts.value('atomic mass constant')/m0 # m_0
waist = 0.012/x0
v_max = 8
z_max = 0.020/x0
z_start = 0.020/x0
dz = 0.0005/x0
dv = 0.0005
t_eval = np.linspace(0.,3000000.,3000001)
omega = 2*np.pi*(cts.c/wavelength) #Transition frequency
Isat = np.pi*cts.h*cts.c*Gamma*2*np.pi*1e6/3*1/(wavelength)**3

# The detunings used in the PRAs:
P = np.array([0.010, 0.050, 0.1, 0.2])
intensities = 2.*P/(np.pi*(0.012)**2)/Isat
power_index = 2

#Make the axis:
zz = np.linspace(-1*z_max,z_max+dz,100)
vv = np.linspace(-1*v_max,v_max+dv,100)
Z,V = np.meshgrid(zz,vv)



H0_X, Bq_X, U_X, Xbasis = pylcp.hamiltonians.XFmolecules.Xstate(B=0,
gamma = 50.697/Gamma,b=154.7/Gamma, c=178.5/Gamma,
    muB = cts.value('Bohr magneton in Hz/T')/1e6*1e-4/Gamma,return_basis=True
    )

# b : SI coupling(isotropic), c : Iz Sz coupling(anisotropic), cc : I N coupling, gamma : S N coupling

E_X = np.unique(np.diag(H0_X))

H0_A, Bq_A, Abasis = pylcp.hamiltonians.XFmolecules.Astate(
    P=+1, Ahfs=-1.5/Gamma, q=0, p=0,gJ=-0.00002,
    muB=cts.value('Bohr magneton in Hz/T')/1e6*1e-4/Gamma, return_basis=True
    )

# gJ : Lande g-factor, p : parity(e parity)

E_A = np.unique(np.diag(H0_A))

dijq = pylcp.hamiltonians.XFmolecules.dipoleXandAstates(
    Xbasis, Abasis, UX=U_X
    )

hamiltonian = pylcp.hamiltonian(H0_X, H0_A, Bq_X, Bq_A, dijq,mass = mass)

In [3]:
def Fixed_detune_MgF_MOT(main_det,det_1,det_2,pr_1,pr_2,laseron,laseroff):
    det_side = det_1/Gamma
    det_side2 = det_2/Gamma
    Avg_X = np.average(E_X)
    
    init_pow = 0.5*2./(np.pi*(0.012)**2)/Isat
    pow_rate1 = pr_1/(2+pr_1)
    pow_rate2 = pr_2/(4+pr_2)
    pow_rate1_i = (1-pow_rate1)/2
    pow_rate2_i = (1-pow_rate2)/4
    
    def Heav_step(t):
        if laseron<=t and t<laseron+14:
            return -1*(t-laseron-7)*((t-laseron-7)**2-49*3)*1/686*1/2+1/2
        elif laseron+14<=t and t<laseroff:
            return 1
        elif t>=laseroff and t<laseroff+14:
            return (t-laseroff-7)*((t-laseroff-7)**2-49*3)*1/686*1/2 + 1/2
        else:
            return 0    
        
    
    laserBeams = pylcp.laserBeams()
# Minus Sideband part
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side-det_side2*2,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side-det_side2*2,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side-det_side2*1,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side-det_side2*1,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side-det_side2*0,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side-det_side2*0,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side+det_side2*1,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side+det_side2*1,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side+det_side2*2,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side+det_side2*2,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i}])

# Main Slowing Laser
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side2*2,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1*pow_rate2_i},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side2*2,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1*pow_rate2_i}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side2*1,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1*pow_rate2_i},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side2*1,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1*pow_rate2_i}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side2*0,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1*pow_rate2},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side2*0,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1*pow_rate2}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side2*1,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1*pow_rate2_i},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side2*1,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1*pow_rate2_i}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side2*2,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1*pow_rate2_i},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side2*2,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1*pow_rate2_i}])
# Plus Sideband part
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side-det_side2*2,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side-det_side2*2,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side-det_side2*1,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side-det_side2*1,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side-det_side2*0,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side-det_side2*0,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side+det_side2*1,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side+det_side2*1,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side+det_side2*2,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side+det_side2*2,
                                     's': lambda R,t : init_pow*np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)*Heav_step(t)*pow_rate1_i*pow_rate2_i}])

    return laserBeams

In [ ]:
def vc_bayesian(main_det,det_1,det_2,pr_1,pr_2,laseron,laseroff):
    laserBeams = Fixed_detune_MgF_MOT()
    magField=lambda R,t : np.zeros(3)
    eqn = pylcp.rateeq(laserBeams=laserBeams,magField=magField,hamitlonian=hamiltonian)

    eqn.set_initial_position_and_velocity(np.array([-1*z_start,-1*z_start,0]),np.array([v0s,v0s,0]))
    eqn.set_initial_pop(np.array([1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0]))

    eqn.evolve_motion([0.,max(t_eval)],t_eval=t_eval,events= None,progress_bar = 0,method='LSODA',max_step=1e5)
    sol = eqn.sol
    
    if abs(sol.r[0][-1])*1000*x0<5 and abs(sol.r[1][-1])*1000*x0<5 and abs(sol.v[0][-1])<5e-2 and abs(sol.v[1][-1])<5e-2:
        return v0s
    else:
        return -1.